In [2]:
from operator import itemgetter
import itertools
from IPython.display import HTML

import pandas as pd
from old_sa import SubstrateAnalyzer
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import pymatgen.core as mg
from pymatgen.core.lattice import Lattice

from pymatgen.io.ase import AseAtomsAdaptor
from ase.db import connect
import glob
import os
import json
from ase import io
from ase.data import chemical_symbols, atomic_numbers

from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d, HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row

import numpy as np
import matplotlib.pyplot as plt

# replace with your API key
mpr = MPRester("API_KEY")

import html5lib
import bs4

from pymatgen.core.composition import Composition

import pickle

# needs mp and oqmd folders on the same layer
dbs = [connect(os.path.abspath('mp/mp_oxides.db')), connect(os.path.abspath('oqmd/oqmd_oxides.db'))]

all_BCC = pickle.load(open("all_BCC.pickle", "rb"))
all_FCC = pickle.load(open("all_FCC.pickle", "rb"))

In [3]:
def make_query_string_from_elements(query_elements):

    symbol_mask = [False for symbol in chemical_symbols]

    for symbol in query_elements: # marks the ones we want
        symbol_mask[atomic_numbers[symbol]] = True

    query_string = ''
    for z, sb in enumerate(symbol_mask):
        if sb:
            query_string += chemical_symbols[z] +'>=0,'
        else:
            query_string += chemical_symbols[z] +'=0,'

    return query_string


def make_alloy_string_from_elements(query_elements):
    qes = sorted(query_elements, key=lambda el: atomic_numbers[el])

    alloy_name = ''
    for el in qes:
        alloy_name+=el

    return alloy_name

In [4]:
def make_spec_query_string_from_elements(query_elements, amounts):
    
    if len(query_elements) != len(amounts):
        print("Invalid inputs!")

    symbol_mask = [False for symbol in chemical_symbols]

    for symbol in query_elements: # marks the ones we want
        symbol_mask[atomic_numbers[symbol]] = True

    query_string = ''
    i = 0
    for z, sb in enumerate(symbol_mask):
        if sb:
            query_string += chemical_symbols[z] +'='+ str(amounts[i]) + ','
            i += 1
        else:
            query_string += chemical_symbols[z] +'=0,'

    return query_string

In [5]:
def groupby_itemkey(iterable, item):
    """groupby keyed on (and pre-sorted by) itemgetter(item)."""
    itemkey = itemgetter(item)
    return itertools.groupby(sorted(iterable, key=itemkey), itemkey)

In [6]:
def MCIA(f, s, conventional = False):
    all_matches = []
    sa = SubstrateAnalyzer()

    if conventional:
        film = SpacegroupAnalyzer(f).get_conventional_standard_structure()
        substrate = SpacegroupAnalyzer(s).get_conventional_standard_structure()
    else:
        film = f
        substrate = s

    # Calculate all matches and group by substrate orientation
    matches_by_orient = groupby_itemkey(
        sa.calculate(film, substrate, lowest = True),
        "sub_miller")

    # Find the lowest area match for each substrate orientation
    lowest_matches = [min(g, key=itemgetter("match_area"))
                      for k, g in matches_by_orient]

    for match in lowest_matches:
        print(match)
        db_entry = {
            "sub_form": substrate.composition.reduced_formula,
            "orient": " ".join(map(str, match["sub_miller"])),
            "film_form": film.composition.reduced_formula,
            "film_orient": " ".join(map(str, match["film_miller"])),
            "area": match["match_area"],
        }
        all_matches.append(db_entry)

    df = pd.DataFrame(all_matches)
    #df.set_index("sub_id", inplace=True)
    return df.sort_values("area")

In [7]:
# query_elements = ['Ni', 'O']
# other_query_elements = ['Ni']
# my_qs = make_query_string_from_elements(query_elements)
# other_my_qs = make_spec_query_string_from_elements(other_query_elements, [1])
# alloy_name = make_alloy_string_from_elements(query_elements)

In [8]:
# howmany = 0
# all_atoms = []
# for db in dbs:
#     for row in db.select(pretty_formula = 'NiO', e_above_hull = 0):
#         print(row.key_value_pairs)
#         print()
#         atoms = row.toatoms()
#         all_atoms.append(atoms)
#         howmany +=1 
# print(howmany)

In [9]:
def get_equiv_alloy_volume(atoms):
    symbols = atoms.get_chemical_symbols()

    volume = 0

    for sym in symbols:
        if sym != 'O':
            volume+=metallic_atomic_volumes[sym]
    return volume

In [10]:
def equiv_alloy_volume(atoms, per_cell_volume):
    symbols = atoms.get_chemical_symbols()
    volume = 0
    for sym in symbols:
        if sym != 'O':
            volume+=per_cell_volume
    return volume

In [11]:
def get_substrate(lattice_param, structure):
    if structure == "BCC":
        return mg.Structure(Lattice.cubic(lattice_param), ["Cu", "Cu"], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif structure == "FCC":
        return mg.Structure(Lattice.cubic(lattice_param), ["Cu", "Cu", "Cu", "Cu"], [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]])

In [12]:
def get_films(metals, per_cell_volume, e_above_hull_lim):
    howmany = 0
    my_qs = make_query_string_from_elements(['O'] + metals) + "e_above_hull<=" + str(e_above_hull_lim)
    substrates = []
    for db in dbs:
        for row in db.select(my_qs):
            atoms = row.toatoms()
            if len(set(atoms.get_chemical_symbols())) == 1:
                continue
            v_mox_ratio = atoms.cell.volume / equiv_alloy_volume(atoms, per_cell_volume)
            try:
                substrates.append({"structure": AseAtomsAdaptor.get_structure(atoms), 
                                   "atoms": atoms,
                                   "source": row['material_id'],
                                   "natoms": row['natoms'],
                                   "spg_num": row['spg_number'],
                                   "formula": row['pretty_formula'],
                                   "ox_at_%": 100*Composition(row['pretty_formula']).get_atomic_fraction("O"),
                                   "e_above_hull": row['e_above_hull'],
                                   "formation_energy_per_atom": row["formation_energy_per_atom"],
                                   "space group": row["spg_symbol"],
                                   "v_mox_ratio": v_mox_ratio,
                                   "ICSD IDs": process_ICSD(row.data.icsd_ids),
                                   "band gap": row["band_gap"]})
            except:
                print('Tabulation failed for:', row)
            howmany +=1
    return substrates

In [13]:
from itertools import compress
def sort_through_dupes(films):
    want = np.ones(len(films), dtype = bool)
    for i in range(len(films)):
        for j in range(i+1, len(films)):
            if (films[i]["formula"] == films[j]["formula"]) and (films[i]["natoms"] == films[j]["natoms"]) and (films[i]["spg_num"] == films[j]["spg_num"]):
                if films[i]["e_above_hull"] < films[j]["e_above_hull"]:
                    want[j] = False
                else:
                    want[i] = False
    return list(compress(films, want))

In [14]:
def print_mat_ids(films):
    for i in range(len(films)):
        print(films[i]["source"])

In [15]:
def process_ICSD(ICSDs):
    def format_int_list(ilist):
        if len(ilist) > 0:
            out_string = '{'
            for iint in ilist[0:-1]:
                out_string+='%i, ' %iint
            out_string+= '%i}'% ilist[-1]
        else:
            out_string = ''
        return out_string

    return format_int_list(sorted(ICSDs))

In [16]:
from pymatgen.analysis.elasticity.strain import Deformation
from pymatgen.core.surface import (SlabGenerator,
                                   get_symmetrically_distinct_miller_indices)
def misfit_strain(film, match, norm = True):
    def fast_norm(a):
        return np.sqrt(np.dot(a, a))
    # need film to be conventional standard structure
    struc = SlabGenerator(film, match['film_miller'], 20, 15,
                              primitive=True).get_slab().oriented_unit_cell

    # Generate 3D lattice vectors for film super lattice
    film_matrix = list(match['film_sl_vecs'])
    film_matrix.append(np.cross(film_matrix[0], film_matrix[1]))

    # Generate 3D lattice vectors for substrate super lattice
    # Out of plane substrate super lattice has to be same length as
    # Film out of plane vector to ensure no extra deformation in that
    # direction
    substrate_matrix = list(match['sub_sl_vecs'])
    temp_sub = np.cross(substrate_matrix[0], substrate_matrix[1])
    temp_sub = temp_sub * fast_norm(film_matrix[2]) / fast_norm(temp_sub)
    substrate_matrix.append(temp_sub)

    transform_matrix = np.transpose(np.linalg.solve(film_matrix,
                                                    substrate_matrix))
    dfm = Deformation(transform_matrix)

    strain = dfm.green_lagrange_strain.convert_to_ieee(struc, initial_fit=False)

    if norm:
        return (np.sum(np.power(np.array(strain), 2))) ** 0.5
    else:
        return strain.von_mises_strain

In [17]:
# conventional substrate, primitive films
def analyze_alloy(metals, composition, structure, e_above_hull_lim = 0.015, no_dupes = True):
    # deal with inputs
    if len(metals) != len(composition):
        print("Inputs do not match!")
        return None
    if round(sum(composition), 6) != 1:
        print("Composition does not add to 1!")
        return None
    parameter = 0
    if structure == "BCC":
        using = all_BCC
        factor = 2
    elif structure == "FCC":
        using = all_FCC
        factor = 4
    for i, metal in enumerate(metals):
        if using.get(metal) is None:
            print("Do not have lattice parameter for:", metal)
            return None
        parameter += using[metal]["val"] * composition[i]
    substrate = SpacegroupAnalyzer(get_substrate(parameter, structure)).get_conventional_standard_structure()
    per_cell_volume = substrate.volume / factor
    films = get_films(metals, per_cell_volume, e_above_hull_lim)
    if no_dupes:
        films = sort_through_dupes(films)
    # !!! actual analyzer part !!!
    all_matches = []
    sa = SubstrateAnalyzer()
    # Calculate all matches and group by substrate orientation
    for f in tqdm(films):
        film = f["structure"]
        matches_by_orient = groupby_itemkey(
            sa.calculate(film, substrate, lowest = True),
            "sub_miller")

        # Find the lowest area match for each substrate orientation
        lowest_matches = [min(g, key=itemgetter("match_area"))
                          for k, g in matches_by_orient]

        for match in lowest_matches:
            try:
                db_entry = {
                    "material ID": f["source"],
                    "oxide": film.composition.reduced_formula,
                    "e_above_hull (eV)": f['e_above_hull'],
                    "form. energy per atom (eV/atom)": f["formation_energy_per_atom"],
                    "space-group": f["space group"],
                    "v_mox_ratio": f["v_mox_ratio"],
                    "atomic % of O in oxide": round(f["ox_at_%"], 2),
                    "band gap": f["band gap"],
                    "alloy orientation": " ".join(map(str, match["sub_miller"])),
                    "oxide orientation": " ".join(map(str, match["film_miller"])),
                    "misfit strain (norm)": misfit_strain(film, match, norm = True),
                    "min. coincident area": match["match_area"],
                    "similar ICSD structure IDs": f["ICSD IDs"],
                }
                all_matches.append(db_entry)
            except:
                print(f["source"], "with composition", film.composition.reduced_formula, "failed")
    df = pd.DataFrame(all_matches)
    df = df.sort_values("min. coincident area")
    print("Calculated parameter is:", parameter)
    print("Found", len(films), "film candidates.")
    return parameter, df

In [28]:
def display_data(df, num_display = None, top_unique = None):
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    if top_unique is not None:
        new_df = pd.concat([df.iloc[:top_unique], ((df.iloc[top_unique:]).drop_duplicates(subset=['material ID']))])
    else:
        new_df = df.copy(deep = True)
    if num_display is None or len(df) < num_display:
        return new_df
    else:
        return new_df.head(num_display)

In [19]:
def formatting(x):
    if "mp" in x: 
        return '<a href="https://materialsproject.org/materials/{0}" target="_blank">{0}</a>'.format(x)
    else: 
        return '<a href="http://oqmd.org/materials/entry/{0}" target="_blank">{1}</a>'.format(x[5:], x)

In [20]:
def make_sortable(name_of_file):
    with open(name_of_file, 'r') as file:
        # read a list of lines into data
        data = file.readlines()

    with open("MCIAheading.txt", 'r') as headingfile:
        # read a list of lines into data
        headingdata = headingfile.readlines()

    for i in range(len(headingdata)):
        data[i] = headingdata[i]

    with open(name_of_file, 'w') as file:
        file.writelines(data)

    f1 = open(name_of_file, 'a+')
    f2 = open("MCIAscript.txt", 'r')

    f1.write(f2.read())

    f1.close()
    f2.close()

In [21]:
from bokeh.layouts import gridplot
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.models.tools import *

def add_plots(df, name_of_pfile, name_of_file, append_string):

    output_file(name_of_pfile)

    source = ColumnDataSource(data=dict(form=list(df["form. energy per atom (eV/atom)"]),
                                        mcia=list(df["min. coincident area"]),
                                        strain=list(df["misfit strain (norm)"]),
                                        names=list(df["oxide"]),
                                        mp_ids=list(df["material ID"]),
                                        ox_orients=list(df["oxide orientation"])))
    
    hover = HoverTool(tooltips=[
        ("mat_id", "@mp_ids"),
        ("oxide orientation", "@ox_orients"),
        ('mcia', '@mcia'),
        ('misfit strain', '@strain'),
        ('formation energy', '@form'),
        ])

    # create a view of the source for one plot to use
    view = CDSView(source=source)

    TOOLS = [BoxZoomTool(), PanTool(), SaveTool(), ResetTool(), hover]

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, title='MCIA vs Formation Energy Per Atom')
    left.circle(x='form', y='mcia', size=8, hover_color="deeppink", source=source)
    left.xaxis[0].axis_label = 'Formation energy per atom (eV/atom)'
    left.yaxis[0].axis_label = 'MCIA'
    
    labels = LabelSet(x='form', y='mcia', text='names',
                  x_offset=3, y_offset=3, source=source, render_mode='canvas', text_font_size="8pt")

    left.add_layout(labels)
   
    # create another new plot, add a renderer that uses the view of the data source
    right = figure(tools=TOOLS, title='Misfit Strain vs Formation Energy Per Atom')
    right.circle(x='form', y='strain', size=8, hover_color="deeppink", source=source, view=view)
    right.xaxis[0].axis_label = 'Formation energy per atom (eV/atom)'
    right.yaxis[0].axis_label = 'Misfit Strain'
    
    labels = LabelSet(x='form', y='strain', text='names',
                  x_offset=3, y_offset=3, source=source, render_mode='canvas', text_font_size="8pt")

    right.add_layout(labels)

    p = gridplot([[left, right]])

    show(p)
    
    f1 = open(name_of_pfile, 'a+')
    f2 = open(name_of_file, 'r')
    
    f1.write("<br>")
    
    f1.write(append_string)
    
    f1.write("<br><br>")

    f1.write(f2.read())

    f1.close()
    f2.close()

In [22]:
def auto_comp(metals):
    return np.ones(len(metals)) / len(metals)

In [23]:
# df stored
#METALS = ["Al", "Co", "Cr", "Fe", "Mn", "Ni", "Ti"]
#METALS = ["Co", "Cr", "Fe", "Mn", "Ni", "Ti"]
#METALS = ["Hf", "Nb", "Ta", "Ti", "Zr", "Al"]
# BCC Hf_0.198_Nb_0.198_Ta_0.198_Ti_0.198_Zr_0.198_Al_0.01
METALS = ["Fe", "Co", "Ni"]
#METALS = ["Cr", "Al", "Fe", "Co", "Ni"]

#COMPS = [0.198, 0.198, 0.198, 0.198, 0.198, 0.01]
#COMPS = auto_comp(METALS)
COMPS = [0.33, 0.33, 0.34]
#COMPS = [0.10, 0.10, 0.26, 0.27, 0.27]

#print(COMPS)
E_HULL = 0.015
STRUCTURE = "BCC"
lattice_param, df = analyze_alloy(METALS, COMPS, STRUCTURE, e_above_hull_lim = E_HULL)

  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

Tabulation failed for: <AtomsRow: formula=Fe4O6, keys=e_above_hull,formation_energy_per_atom,material_id,spg_number,spg_symbol,pretty_formula>


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:51<00:00,  1.08s/it]

Calculated parameter is: 2.8255107360359517
Found 48 film candidates.


In [29]:
dfcopy = display_data(df, top_unique = 5)
dfcopy2 = display_data(df, top_unique = 5)

In [30]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [31]:
# create html out of the df
calc_param = lattice_param
dfcopy['material ID'] = dfcopy['material ID'].apply(formatting)
specific_part = STRUCTURE + "-" + "+".join("{0}({1})".format(i, round(j, 3)) for i, j in zip(METALS, COMPS)) + "e_lim=" + str(E_HULL) + ".html"

# can change these names 
# name_of_file = 'MCIAhtmlNew/' + specific_part
# name_of_pfile = 'MCIAwPlots/' + specific_part
name_of_file = 'noplot' + specific_part
name_of_pfile = 'plot' + specific_part

dfcopy.to_html(open(name_of_file, 'w'), 
                    justify = "center", 
                    table_id = "myTable2",
                    render_links=True, 
                    escape = False)
make_sortable(name_of_file)
append_string = STRUCTURE + "-" + "_".join("{0}({1})".format(i, round(j, 3)) for i, j in zip(METALS, COMPS)) + ": a = " + str(round(calc_param, 4)) + " Ang"
add_plots(dfcopy2, name_of_pfile, name_of_file, append_string)

In [32]:
# showing top 5 and only unique films afterwards
new = display_data(df, top_unique = 5)
display(new)

,material ID,oxide,e_above_hull (eV),form. energy per atom (eV/atom),space-group,v_mox_ratio,atomic % of O in oxide,band gap,alloy orientation,oxide orientation,misfit strain (norm),min. coincident area,similar ICSD structure IDs
48,mp-1283043,CoO2,0.007579,-1.073220,P6_3/mmc,3.171284,66.67,0.0000,1 0 0,0 1 1,0.009766,32.098462,
75,oqmd-4562,Co3O4,0.000000,-1.343081,Fd-3m,1.940207,57.14,1.7960,1 0 0,1 0 -1,0.013326,32.547442,{24210}
69,mp-32686,CoO2,0.008335,-1.072464,P-3m1,2.981353,66.67,0.0000,1 0 0,0 1 1,0.021526,38.980779,"{26763, 53994, 88722}"
63,mp-1272749,CoO2,0.006609,-1.074190,R-3m,4.268156,66.67,0.0000,1 0 0,0 0 1,0.013677,39.342687,{20566}
97,oqmd-1219293,CoO,0.000000,-1.254691,F-43m,2.129886,50.00,0.8730,1 0 0,1 1 0,0.034387,41.958003,
98,oqmd-1219293,CoO,0.000000,-1.254691,F-43m,2.129886,50.00,0.8730,1 1 0,1 0 -1,0.010461,44.503183,
64,mp-1272749,CoO2,0.006609,-1.074190,R-3m,4.268156,66.67,0.0000,1 1 0,0 1 -1,0.006960,45.427633,{20566}
76,oqmd-4562,Co3O4,0.000000,-1.343081,Fd-3m,1.940207,57.14,1.7960,1 1 0,1 1 -1,0.013326,46.029033,{24210}
15,mp-656887,Ni3O4,0.000000,-1.046750,Cmmm,2.103674,57.14,0.3954,1 0 0,0 1 1,0.032038,47.581173,
138,oqmd-1444447,NiO,0.002362,-1.239151,P-1,1.605037,50.00,2.7010,1 1 1,1 0 0,0.036150,52.632568,
